In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [9]:
SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : alpha*c})

In [10]:
SE9.eckardt_points_labels

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G5', 'F35'],
 ['E4', 'G6', 'F46'],
 ['E5', 'G2', 'F25'],
 ['E6', 'G1', 'F16'],
 ['F12', 'F36', 'F45'],
 ['F13', 'F24', 'F56'],
 ['F15', 'F26', 'F34']]

In [11]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 5.92 s, sys: 381 ms, total: 6.3 s
Wall time: 53.2 s


In [12]:
len(adm_SE9)

1296

In [13]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 1.62 s, sys: 202 ms, total: 1.82 s
Wall time: 4.5 s


In [14]:
len(simm_SE9)

54

In [15]:
G_SE9 = Group(simm_SE9)

In [16]:
G_SE9.get_order()

54

In [17]:
G_SE9.is_abelian()

False

In [18]:
G_SE9.get_divisors_of_order()

[1, 2, 3, 6, 9, 18, 27, 54]

In [19]:
#G_SE9.get_order_of_elements()

In [20]:
#[G_SE9.apply_to(p) for p in SE9.eckardt_points]

In [21]:
#Eckardt points

In [22]:
perms = []
for el in G_SE9:
    perms.append(Permutation(apply_proj_to_eck(el, SE9.eckardt_points)).to_cycles())

In [23]:
PermutationGroup(perms).order()

18

In [24]:
PermutationGroup(perms).structure_description()

'(C3 x C3) : C2'

In [25]:
#lines

In [26]:
adm_perm = SE9.find_admissible_permutations()
keys = list(SE9.cl_lines.keys())

In [41]:
perms_lines = []
for simm in simm_SE9:
    perms_lines.append(Permutation(apply_proj_to_lines(simm, list(SE9.cl_lines.values()))).to_cycles())

In [32]:
Gr = PermutationGroup(perms_lines)

In [33]:
Gr.order()

54

In [34]:
Gr.structure_description()

'((C3 x C3) : C3) : C2'

In [35]:
Gr.minimal_generating_set()

[(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (1,9,14)(2,19,10)(3,13,4)(5,21,17)(6,20,16)(7,8,22)(11,24,26)(12,23,25)(15,18,27),
 (1,21,3)(2,7,13)(4,27,20)(5,26,19)(6,24,9)(8,12,14)(10,16,23)(11,15,22)(17,25,18)]

In [36]:
nor_sub = Gr.normal_subgroups()
[el.order() for el in nor_sub]

[54, 27, 9, 9, 9, 9, 3, 1]

In [37]:
nor_sub_sub = nor_sub[1].normal_subgroups()
[el.order() for el in nor_sub_sub]

[27, 9, 9, 9, 9, 3, 1]

In [38]:
lines_three_eck = []
for i in range(7):
    for j in range(i+1, 8):
        for k in range(j+1, 9):
            point1 = SE9.eckardt_points[i]
            point2 = SE9.eckardt_points[j]
            point3 = SE9.eckardt_points[k]   
            line12_pl = Point(matrix([point1, point2]).minors(2))
            line23_pl = Point(matrix([point2, point3]).minors(2))
            if line12_pl == line23_pl and line12_pl not in [line.plucker for line in lines_three_eck]:
                planes = get_planes(line12_pl)
                lines_three_eck.append(Line(planes,[point1, point2], line12_pl))

In [39]:
[[line1.are_incident(line2) for line2 in lines_three_eck] for line1 in lines_three_eck]

[[True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 [True, True, True, True, True, True, True, True, True, True, True, True]]

In [40]:
get_plane_containing_two_incident_lines(lines_three_eck[0], lines_three_eck[1])

x + (-1/4*alpha - 1/4)*y + (1/4*alpha + 1/4)*z